In [66]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.path as mpath
import random

# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import random
import numba
from numba import jit

%matplotlib qt

In [65]:
def plot_simulation(ys, dt=0.01, num_steps=2000):

    # set some local vars
    num_svs = ys.shape[1]
    xvals = np.arange(num_steps, step=dt)
    plot_offset = num_svs-1

    # plot overlapping line plot of each state variable
    plt.figure()
    ax1 = plt.subplot(211)
    ax2 = plt.subplot(212)

    # iterate state variables and plot
    for sv in range(num_svs):

        # plot first plot
        to_plot = ys[:,sv]
        ax1.plot(xvals, to_plot, label='x{}'.format(sv))

        # plot second
        to_plot2 = (to_plot - np.min(to_plot)) / (np.max(to_plot) - np.min(to_plot))
        ax2.plot(xvals, to_plot2 + plot_offset, label='x{}'.format(sv))

        plot_offset -= 1
    
    # decorate ax1
    ax1.legend(loc='upper left')
    ax1.set_xlabel('t')
    ax1.set_ylabel('y')

    # decorate ax2
    ax2.legend(loc='upper left')
    ax2.set_xlabel('t')
    ax2.get_yaxis().set_visible(False)


def colorline(
    x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0),
        linewidth=3, alpha=1.0):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, norm=norm,
                              linewidth=linewidth, alpha=alpha)

    ax = plt.gca()
    ax.add_collection(lc)
    ax.set_xlim(x.min() - 0.1 * x.max(), x.max() + 0.1*x.max())
    ax.set_ylim(ys.min() - 0.1 * y.max(), ys.max() + 0.1 * y.max())

    return lc


def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

    
def f(ys_i, ode_ops={}):

    # set local var names for fxn
    x = ys_i[0]
    y = ys_i[1]
    
    # do something
    s=10 
    r=28 
    b=2.667
    x_dot = s*(y - x)
    y_dot = r*x - y

    # repack and return
    return np.array([x_dot, y_dot])

# Simple function for euler approximation of dyns
def run_simulation(ics, ode=f, dt=0.01, num_steps=2000, ode_ops={}):

    # initialize output vector
    num_svs = len(ics)
    ys = np.zeros((int(num_steps/dt), num_svs))
    ys[:] = np.NaN

    # Set initial values
    ys[0, :] = ics

    # Step through "time", calculating the partial derivatives at the current point
    # and using them to estimate the next point
    for i in range(ys.shape[0]-1):

        # calculate next step
        y_dot = ode(ys[i, :], ode_ops=ode_ops)
        ys[i + 1, :] = ys[i, :] + (y_dot * dt)

        # printout on 10% progress
        #if i % (ys.shape[0]//10) == 0:
        #    print('Simulating step {}'.format(i))
    
    return ys

# test quick simulation with random system
dt = 0.01
num_steps = 10
ode_ops = {}
ics = np.array([0.5, 0.5])
ys = run_simulation(ics, dt=dt, num_steps=num_steps)
plot_simulation(ys, dt=dt, num_steps=num_steps)

In [25]:
# modle for single harmonic oscillator
def harmonic_oscillator_ode(ys_i, ode_ops={}):

    vt_old = ys_i[0]
    xt_old = ys_i[1]
    k = ode_ops['k']
    m = ode_ops['m']

    # simple harmonic oscillator as two coupled odes
    dvdt = (-k/m)*xt_old
    dxdt = vt_old

    return np.array([dvdt, dxdt])


ics = [0, 1]
ode_ops = {'k': 1, 'm': 1}
num_steps = 100
dt = 0.001
ys = run_simulation(ics, ode=harmonic_oscillator_ode, dt=dt, num_steps=num_steps, ode_ops=ode_ops)
plot_simulation(ys, dt=dt, num_steps=num_steps)

Simulating step 0
Simulating step 10000
Simulating step 20000
Simulating step 30000
Simulating step 40000
Simulating step 50000
Simulating step 60000
Simulating step 70000
Simulating step 80000
Simulating step 90000


In [67]:
# modle for coupled harmonic oscillators
def coupled_oscillator_ode(ys_i, ode_ops={}):

    # unpack local vars
    v1_old = ys_i[0]
    x1_old = ys_i[1]
    v2_old = ys_i[2]
    x2_old = ys_i[3]
    k = ode_ops['k']
    kprime = ode_ops['kprime']
    m = ode_ops['m']

    # two identical harmonic oscillators, coupled by an additional spring
    dv1 = (-(k + kprime) * x1_old + kprime*x2_old)/m
    dx1 = v1_old
    dv2 = (-(k + kprime) * x2_old + kprime*x1_old)/m
    dx2 = v2_old

    return np.array([dv1, dx1, dv2, dx2])


ics = [0, 10, 0, 3]
ode_ops = {'k': 1, 'm': 2, 'kprime': 5}
num_steps = 100
dt = 0.001
ys = run_simulation(ics, ode=coupled_oscillator_ode, dt=dt, num_steps=num_steps, ode_ops=ode_ops)
plot_simulation(ys[:, [1,3]], dt=dt, num_steps=num_steps)


In [69]:
# get random values for simulations
iters = 10
num_steps = 1000
dt = 0.001

# holders
dv1_list = []
dx1_list = []
dv2_list = []
dx2_list = []
k_list = []
m_list = []
kprime_list = []
ys_system = np.zeros((int(num_steps/dt), iters*2))
unit_counter = 0

# iteratively simulate and grab some neurons
for i in range(iters):

    # grab random initial conditions and params
    dv1_0 = 0
    dx1_0 = random.randint(0, 50)
    dv2_0 = 0
    dx2_0 = random.randint(50, 100)
    k_0 = random.uniform(0.1, 3)
    m_0 = random.uniform(1, 10)
    kprime_0 = random.uniform(0.1, 4)

    # store
    dv1_list.append(dv1_0)
    dx1_list.append(dx1_0)
    dv2_list.append(dv2_0)
    dx2_list.append(dx2_0)
    k_list.append(k_0)
    m_list.append(m_0)
    kprime_list.append(kprime_0)

    # pack
    ics = [dv1_0, dx1_0, dv2_0, dx2_0]
    ode_ops = {'k': k_0, 'm': m_0, 'kprime': kprime_0}

    # run sim
    ys = run_simulation(ics, ode=coupled_oscillator_ode, dt=dt, num_steps=num_steps, ode_ops=ode_ops)
    
    # store positions
    ys_system[:, unit_counter:unit_counter+2] = ys[:, [1,3]]

    # increment index holder
    unit_counter += 2

    # printout
    print('Completed iteration: {}'.format(i))

# plot it all
plot_simulation(ys_system, dt=dt, num_steps=num_steps)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9


In [70]:
k

NameError: name 'k' is not defined